In [5]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

In [6]:
# 读取player 的数据
player = pd.read_csv("../datafrom200/players.csv")
x_list = player.columns[7:-5]
x_list = ['Crossing', 'Finishing', 'Heading_Accuracy', 'Short_Passing', 'Volleys',
       'Dribbling', 'Curve', 'FK_Accuracy', 'Long_Passing', 'Ball_Control',
       'Acceleration', 'Sprint_Speed', 'Agility', 'Reactions', 'Balance',
       'Shot_Power', 'Jumping', 'Stamina', 'Strength', 'Long_Shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive_Awareness']
print(x_list)
X = player[x_list]
y = player["value"]
len(x_list)

['Crossing', 'Finishing', 'Heading_Accuracy', 'Short_Passing', 'Volleys', 'Dribbling', 'Curve', 'FK_Accuracy', 'Long_Passing', 'Ball_Control', 'Acceleration', 'Sprint_Speed', 'Agility', 'Reactions', 'Balance', 'Shot_Power', 'Jumping', 'Stamina', 'Strength', 'Long_Shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Defensive_Awareness']


27

In [7]:
# 划分训练集 train, validation, test : 0.7, 0.15, 0.15
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.3, random_state=21)
X_validation, X_test, y_validation, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=23)

In [8]:
# Adaboost
from sklearn.ensemble import AdaBoostRegressor
i = 10
clf = AdaBoostRegressor(n_estimators=i)
clf.fit(X_train, y_train)
print("R", clf.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(clf.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(clf.predict(X_validation), y_validation))
print("RMSLE", mean_squared_log_error(clf.predict(X_validation), y_validation))

R 0.7729781975502361
RMSE 4286431.008031562
MAE 2043267.1213696746
RMSLE 1.5085553838770587


In [9]:
# XGBoost
import xgboost as xgb
n, depth = 60, 20
bst = xgb.XGBRFRegressor(n_estimators=n, max_depth=depth)
bst.fit(X_train, y_train)
print("R", bst.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(bst.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(bst.predict(X_validation), y_validation))
print("RMSLE", mean_squared_log_error(bst.predict(X_validation), y_validation))

R 0.8849238797563104
RMSE 3051790.1005353173
MAE 918701.5076130972
RMSLE 0.15975791462144148


In [15]:
import lightgbm as lgb
leaves, depth, n = 31, 19, 500
cbr = lgb.LGBMRegressor(num_leaves=leaves, max_depth=depth, n_estimators=n)
cbr.fit(X_train, y_train)
print("R", cbr.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(cbr.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(cbr.predict(X_validation), y_validation))
# print("RMSLE", mean_squared_log_error(cbr.predict(X_validation), y_validation))

R 0.8959461903429242
RMSE 2901957.4943126254
MAE 876591.2842372275


ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [11]:
# 随机森林
from sklearn.ensemble import RandomForestRegressor
n = 80
clf = RandomForestRegressor(n_estimators=n)
clf.fit(X_train, y_train)
print("R", clf.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(clf.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(clf.predict(X_validation), y_validation))
print("RMSLE", mean_squared_log_error(clf.predict(X_validation), y_validation))

R 0.8773490265124603
RMSE 3150631.088339888
MAE 973366.28763089
RMSLE 0.16433719817842307


In [12]:
# GBDT
from sklearn.ensemble import GradientBoostingRegressor
n = 600
reg = GradientBoostingRegressor(n_estimators=n)
reg.fit(X_train, y_train)
print("R", reg.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(reg.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(reg.predict(X_validation), y_validation))
# print("RMSLE", mean_squared_log_error(reg.predict(X_validation), y_validation))

R 0.9032319116031758
RMSE 2798518.1968447655
MAE 949712.6579044838


In [13]:
# Catboost
import catboost
reg = catboost.CatBoostRegressor()
reg.fit(X_train, y_train)
print("R", reg.score(X_validation, y_validation))
print("RMSE", np.sqrt(mean_squared_error(reg.predict(X_validation), y_validation)))
print("MAE", mean_absolute_error(reg.predict(X_validation), y_validation))
# print("RMSLE", mean_squared_log_error(reg.predict(X_validation), y_validation))

Learning rate set to 0.055843
0:	learn: 8997539.2573857	total: 58.5ms	remaining: 58.4s
1:	learn: 8691234.3774489	total: 60.3ms	remaining: 30.1s
2:	learn: 8404536.4388329	total: 62.7ms	remaining: 20.8s
3:	learn: 8137637.8349213	total: 64.6ms	remaining: 16.1s
4:	learn: 7880035.9396335	total: 66.3ms	remaining: 13.2s
5:	learn: 7639571.7724194	total: 68.5ms	remaining: 11.4s
6:	learn: 7405464.0669408	total: 71.8ms	remaining: 10.2s
7:	learn: 7191877.9500739	total: 73.9ms	remaining: 9.16s
8:	learn: 6973833.3564897	total: 75.5ms	remaining: 8.31s
9:	learn: 6776354.2619976	total: 77.4ms	remaining: 7.66s
10:	learn: 6596944.9895090	total: 79ms	remaining: 7.1s
11:	learn: 6424378.8032592	total: 80.8ms	remaining: 6.66s
12:	learn: 6247717.0948001	total: 82.6ms	remaining: 6.27s
13:	learn: 6089135.3290878	total: 84.1ms	remaining: 5.92s
14:	learn: 5945623.5949379	total: 86.3ms	remaining: 5.67s
15:	learn: 5801071.8244718	total: 87.9ms	remaining: 5.41s
16:	learn: 5652974.3763832	total: 89.5ms	remaining: 5.1

In [14]:
# list_adaboost # adaboost: 10, 0.6945326685800794
# n_list_lbm = np.array(list_lightgbm) # leaves, depth, n, score : 31, 19, 500, 0.8007898902056965
# list_randomforest # [80, 0.7839175337533079]
# list_gbdt # [n = 600, 0.8264955831199144]
# catboost  defalt: 0.8027687798692258
# list_xgboost # [60, 20, 0.7812782293507701]

# Adaboost
 0.7678737783884331 R    

 4334351.772382652 RMSE 

2034372.1610311233 MAE   

 1.4164760588279637 RMSLE

# Xgboost
0.8868637133128766 R

3025958.826846494 RMSE 

923966.9140471614 MAE 

0.15334408868795865 RMSLE

# lgb
0.8948018678333571 R 

2917870.8720710855 RMSE 

863827.241753701 MAE 

# 随机森林
0.8810732244650263 R 

3102429.1779398853 RMSE 

971012.3118455497 MAE 

0.15934332311326996 RMSLE 

# GBDT
0.900796741780507 R 

2833511.7791834464 RMSE 

948986.9158787137 MAE 

# Catboost
0.8959925992883182 R 

2901310.272421131 RMSE 

879135.1904306137 MAE 